In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pymc3 as pm
import numpy as np
import  pandas as pd

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,log_loss
from sklearn.model_selection import train_test_split
from pandas_ml import ConfusionMatrix as cm
from sklearn.preprocessing import StandardScaler
from sklearn import svm

from sklearn.neighbors.kde import KernelDensity

from IPython.display import display_html

from scipy.stats import norm

import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
import statsmodels as sm
import matplotlib

from IPython.display import display,clear_output,HTML

In [2]:
#### Source= https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side

def mydisplay(dfs, names=[]):
    html_str = ''
    if names:
        html_str += ('<tr>' + 
                     ''.join(f'<td style="text-align:center">{name}</td>' for name in names) + 
                     '</tr>')
    html_str += ('<tr>' + 
                 ''.join(f'<td style="vertical-align:top"> {df.to_html(index=False)}</td>' 
                         for df in dfs) + 
                 '</tr>')
    html_str = f'<table>{html_str}</table>'
    html_str = html_str.replace('table','table style="display:inline"')
    display_html(html_str, raw=True)

# Predictions for January

In [6]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

jan=pd.read_csv('Feature-Set_Jan_Final.csv',index_col='User')
print('\nOriginal Jan data\nInfected/Unifected Users')
display(jan.Target.value_counts())
user_jan=jan.index.values

no=[user for user in user_jan if user not in user_Dec]

o=[user for user in user_jan if user in user_Dec]

jan.drop(o,inplace=True)

print("\nAfter eliminating overlapping users:")
print(jan.Target.value_counts())

# x=jan[jan.Target==1]
# y=jan[jan.Target==0]
# y=y.sample(n=532,random_state=10)
# z=x.append(y)
z=jan.sample(frac=1,random_state=10)
Y_test=z.Target.values

Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Jan data
Infected/Unifected Users


0    6406
1     532
Name: Target, dtype: int64


After eliminating overlapping users:
0    6181
1     532
Name: Target, dtype: int64


## Individual features

### Non-parametric approach

In [8]:
lst=[]
for col in train.columns[:-1]:
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

    df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
    df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    mydisplay((pd.crosstab(df.Non_Mal,df.Ground_Truth),pd.crosstab(df.Mal,df.Ground_Truth)),names=['Non-Malicious','Malicious'])
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/OneClassSVM/Jan/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Non-Malicious,Malicious
Malicious,Non_Malicious
94,229
438,5952
Malicious,Non_Malicious
410,2177
122,4004


-----------------------------------------------------------------------------------------------------
DP



Non-Malicious,Malicious
Malicious,Non_Malicious
56,556
476,5625
Malicious,Non_Malicious
241,966
291,5215


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Non-Malicious,Malicious
Malicious,Non_Malicious
120,139
412,6042
Malicious,Non_Malicious
444,2853
88,3328


-----------------------------------------------------------------------------------------------------
2300



Non-Malicious,Malicious
Malicious,Non_Malicious
115,122
417,6059
Malicious,Non_Malicious
432,2840
100,3341


-----------------------------------------------------------------------------------------------------


#### Non-Parametric Test w/kernelDensity 

https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity

https://sebastianraschka.com/faq/docs/parametric_vs_nonparametric.html

In [9]:
for col in train.columns[:-1]:
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
    df['Factor']=df['Non_Mal']/df['Mal']
    df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Ground_Truth,df.Factor))
    print("-----------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/KDE/Jan/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,294,238
Non_Malicious,816,5365


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
DP



Factor,Mal
Ground_Truth,
Malicious,532
Non_Malicious,6181


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2200



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,327,205
Non_Malicious,1153,5028


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2300



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,297,235
Non_Malicious,1008,5173


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


In [10]:
col=train.columns[:-1]

## Combined features using OneClassSVM

In [11]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
non_mal_lst=clf.predict(X_test)

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
mal_lst=clf.predict(X_test)


df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
df.set_index(z.index.values,drop=True,inplace=True)
df.head()

,Non_Mal,Mal,Ground_Truth
741124bc349936f0b0a6f98371cda1ae3a237cfa8c77a90501796ae968b9f1f0,Non_Malicious,Malicious,Non_Malicious
3c9a19258e0b785c33db19461f6796d20b0e672af5631d5695079317629051f2,Non_Malicious,Malicious,Non_Malicious
bb2d29b917d20f2884afc3a4fedd9a1740e313610f02cb41126366bff392ee22,Malicious,Malicious,Non_Malicious
ce9ff3751af4593fba8da405ecc3453020e010fad7e6b87570a424e184aceb6f,Non_Malicious,Malicious,Non_Malicious
5536f91e3ebd4b7c05ac5d559ee3762af73b9ee7e717cc39d8637cdcb9102da7,Non_Malicious,Malicious,Non_Malicious


In [12]:
print('Model built on Non_malicious Users')
display((pd.crosstab(df.Non_Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------\n\n")

print('Model built on Malicious Users')
display((pd.crosstab(df.Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------")

file_name='Non-parametric/OneClassSVM/Jan/Non_para_combined.csv'
df.to_csv(file_name)

Model built on Non_malicious Users


Ground_Truth,Malicious,Non_Malicious
Non_Mal,,
Malicious,226,600
Non_Malicious,306,5581


-----------------------------------------------------------------------------------------------------


Model built on Malicious Users


Ground_Truth,Malicious,Non_Malicious
Mal,,
Malicious,414,5081
Non_Malicious,118,1100


-----------------------------------------------------------------------------------------------------


## Combined Features using KDE

In [13]:
col=train.columns[:-1]

In [14]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

non_mal=np.exp(kde.score_samples(X_test))

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

mal=np.exp(kde.score_samples(X_test))

df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
df['Factor']=df['Non_Mal']/df['Mal']
df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(pd.crosstab(df.Ground_Truth,df.Factor))
print("-----------------------------------------------------------------------------------------------------")
print("-----------------------------------------------------------------------------------------------------")
file_name='Non-parametric/KDE/Jan/Non_para_combined.csv'

df.set_index(z.index.values,drop=True,inplace=True)

df.to_csv(file_name)

Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,407,125
Non_Malicious,2771,3410


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


# For February

In [3]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

Feb=pd.read_csv('Feature-Set_Feb_Final.csv',index_col='User')
print('\nOriginal Feb data\nInfected/Unifected Users')
display(Feb.Target.value_counts())
user_Feb=Feb.index.values

no=[user for user in user_Feb if user not in user_Dec]

o=[user for user in user_Feb if user in user_Dec]

Feb.drop(o,inplace=True)

print("\nAfter eliminating overlapping users:")
print(Feb.Target.value_counts())

# x=Feb[Feb.Target==1]
# y=Feb[Feb.Target==0]
# y=y.sample(n=532,random_state=10)
# z=x.append(y)
z=Feb.sample(frac=1,random_state=10)
Y_test=z.Target.values

Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Feb data
Infected/Unifected Users


0    6125
1    1131
Name: Target, dtype: int64


After eliminating overlapping users:
0    5899
1    1131
Name: Target, dtype: int64


## Individual features

### Non-parametric approach

In [4]:
lst=[]
for col in train.columns[:-1]:
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

    df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
    df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    mydisplay((pd.crosstab(df.Non_Mal,df.Ground_Truth),pd.crosstab(df.Mal,df.Ground_Truth)),names=['Non-Malicious','Malicious'])
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/OneClassSVM/Feb/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Non-Malicious,Malicious
Malicious,Non_Malicious
140,228
991,5671
Malicious,Non_Malicious
965,2869
166,3030


-----------------------------------------------------------------------------------------------------
DP



Non-Malicious,Malicious
Malicious,Non_Malicious
113,529
1018,5370
Malicious,Non_Malicious
588,1445
543,4454


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Non-Malicious,Malicious
Malicious,Non_Malicious
187,151
944,5748
Malicious,Non_Malicious
947,2931
184,2968


-----------------------------------------------------------------------------------------------------
2300



Non-Malicious,Malicious
Malicious,Non_Malicious
168,146
963,5753
Malicious,Non_Malicious
932,2885
199,3014


-----------------------------------------------------------------------------------------------------


#### Non-Parametric Test w/kernelDensity 

https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity

https://sebastianraschka.com/faq/docs/parametric_vs_nonparametric.html

In [5]:
for col in train.columns[:-1]:
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
    df['Factor']=df['Non_Mal']/df['Mal']
    df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Ground_Truth,df.Factor))
    print("-----------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/KDE/Feb/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,767,364
Non_Malicious,1711,4188


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
DP



Factor,Mal
Ground_Truth,
Malicious,1131
Non_Malicious,5899


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2200



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,825,306
Non_Malicious,1873,4026


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2300



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,756,375
Non_Malicious,1692,4207


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


In [6]:
col=train.columns[:-1]

## Combined features using OneClassSVM

In [7]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
non_mal_lst=clf.predict(X_test)

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
mal_lst=clf.predict(X_test)


df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
df.set_index(z.index.values,drop=True,inplace=True)
df.head()

,Non_Mal,Mal,Ground_Truth
d3a32a43f1ec2fe0d8f3d94aa41ea08f6b76676a984c0c3c8aa64c02613084ec,Non_Malicious,Malicious,Non_Malicious
a42ad28865008e36f8448efe8b29621b7fb81becccbc22cf84b61593f709602f,Non_Malicious,Malicious,Non_Malicious
411edeb48382a082e760415372e2d7e6e9f15d680b444b836b7b062f8cbeca2b,Non_Malicious,Malicious,Non_Malicious
1e4ee3741cc8bbb7f66dbfd9b0d00ef6006ea1144bbc3e6e99800a9eb68001d9,Non_Malicious,Malicious,Malicious
1d9c89bccbd57c90ad801f11ad4f0f8ba008db2230e6236176506e533d590836,Non_Malicious,Non_Malicious,Non_Malicious


In [8]:
print('Model built on Non_malicious Users')
display((pd.crosstab(df.Non_Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------\n\n")

print('Model built on Malicious Users')
display((pd.crosstab(df.Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------")

file_name='Non-parametric/OneClassSVM/Feb/Non_para_combined.csv'
df.to_csv(file_name)

Model built on Non_malicious Users


Ground_Truth,Malicious,Non_Malicious
Non_Mal,,
Malicious,367,611
Non_Malicious,764,5288


-----------------------------------------------------------------------------------------------------


Model built on Malicious Users


Ground_Truth,Malicious,Non_Malicious
Mal,,
Malicious,965,5073
Non_Malicious,166,826


-----------------------------------------------------------------------------------------------------


## Combined Features using KDE

In [9]:
col=train.columns[:-1]

In [10]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

non_mal=np.exp(kde.score_samples(X_test))

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

mal=np.exp(kde.score_samples(X_test))

df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
df['Factor']=df['Non_Mal']/df['Mal']
df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(pd.crosstab(df.Ground_Truth,df.Factor))
print("-----------------------------------------------------------------------------------------------------")
print("-----------------------------------------------------------------------------------------------------")
file_name='Non-parametric/KDE/Feb/Non_para_combined.csv'

df.set_index(z.index.values,drop=True,inplace=True)

df.to_csv(file_name)

Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,947,184
Non_Malicious,3312,2587


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


# For March

In [3]:
train=pd.read_csv('Feature_Vector_50_50_12_24_random_state_0.csv',index_col='User')
print('Training Set\nInfected/Unifected Users')
print()
display(train.Target.value_counts())
user_Dec=train.index.values

Mar=pd.read_csv('Feature-Set_Mar.csv',index_col='User')
print('\nOriginal Mar data\nInfected/Unifected Users')
display(Mar.Target.value_counts())
user_Mar=Mar.index.values

no=[user for user in user_Mar if user not in user_Dec]

o=[user for user in user_Mar if user in user_Dec]

Mar.drop(o,inplace=True)


x=Mar[Mar.Target==1]
y=Mar[Mar.Target==0]
y=y.sample(n=4787,random_state=10)
z=x.append(y)
z=z.sample(frac=1,random_state=10)
Y_test=z.Target.values
print("\nAfter eliminating overlapping users:")
print(z.Target.value_counts())


Training Set
Infected/Unifected Users



1    2910
0    2910
Name: Target, dtype: int64


Original Mar data
Infected/Unifected Users


0    54891
1     5931
Name: Target, dtype: int64


After eliminating overlapping users:
1    4787
0    4787
Name: Target, dtype: int64


## Individual features

### Non-parametric approach

In [4]:
lst=[]
for col in train.columns[:-1]:
    ### Non-Malicious Users
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal_lst=clf.predict(X_test)
    
    ### Malicious users
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
    clf.fit(scaled_X)
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal_lst=clf.predict(X_test)
    df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

    df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
    df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    print('\033[1m' + str(col) + '\033[0m\n')
    mydisplay((pd.crosstab(df.Non_Mal,df.Ground_Truth),pd.crosstab(df.Mal,df.Ground_Truth)),names=['Non-Malicious','Malicious'])
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/OneClassSVM/Mar/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Non-Malicious,Malicious
Malicious,Non_Malicious
507,344
4280,4443
Malicious,Non_Malicious
3661,1722
1126,3065


-----------------------------------------------------------------------------------------------------
DP



Non-Malicious,Malicious
Malicious,Non_Malicious
464,604
4323,4183
Malicious,Non_Malicious
3106,1290
1681,3497


-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
2200



Non-Malicious,Malicious
Malicious,Non_Malicious
590,104
4197,4683
Malicious,Non_Malicious
3319,1718
1468,3069


-----------------------------------------------------------------------------------------------------
2300



Non-Malicious,Malicious
Malicious,Non_Malicious
541,92
4246,4695
Malicious,Non_Malicious
3275,1701
1512,3086


-----------------------------------------------------------------------------------------------------


#### Non-Parametric Test w/kernelDensity 

https://jakevdp.github.io/PythonDataScienceHandbook/05.13-kernel-density-estimation.html

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html#sklearn.neighbors.KernelDensity

https://sebastianraschka.com/faq/docs/parametric_vs_nonparametric.html

In [5]:
for col in train.columns[:-1]:
    X_train=train[train.Target==0][col].values
    Y_train=train[train.Target==0]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    non_mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    X_train=train[train.Target==1][col].values
    Y_train=train[train.Target==1]['Target'].values
    scaler=StandardScaler()
    scaled_X=scaler.fit_transform(X_train.reshape(-1,1))
    kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X.reshape(-1,1))
    X_test=scaler.transform(z[col].values.reshape(-1,1))
    mal=np.exp(kde.score_samples(X_test.reshape(-1,1)))
    
    df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
    df['Factor']=df['Non_Mal']/df['Mal']
    df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
    df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
    
    print('\033[1m' + str(col) + '\033[0m\n')
    display(pd.crosstab(df.Ground_Truth,df.Factor))
    print("-----------------------------------------------------------------------------------------------------")
    print("-----------------------------------------------------------------------------------------------------")
    file_name='Non-parametric/KDE/Mar/Non_para_col_'+str(col)+'.csv'
    df.to_csv(file_name)

SP



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,2942,1845
Non_Malicious,974,3813


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
DP



Factor,Mal
Ground_Truth,
Malicious,4787
Non_Malicious,4787


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
NAPP



...

-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2200



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,2723,2064
Non_Malicious,963,3824


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------
2300



Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,2593,2194
Non_Malicious,840,3947


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


In [6]:
col=train.columns[:-1]

## Combined features using OneClassSVM

In [7]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
non_mal_lst=clf.predict(X_test)

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(scaled_X)
X_test=scaler.transform(z[col].values)
mal_lst=clf.predict(X_test)


df=pd.DataFrame({'Non_Mal':non_mal_lst.reshape(1,-1)[0].tolist(),'Mal':mal_lst.reshape(1,-1)[0].tolist(),
              'Ground_Truth':Y_test.reshape(1,-1)[0].tolist()},index=z.index.values)

df['Non_Mal']=df['Non_Mal'].apply(lambda x:'Non_Malicious' if x==1 else 'Malicious')
df['Mal']=df['Mal'].apply(lambda x:'Malicious' if x==1 else 'Non_Malicious')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')
df.set_index(z.index.values,drop=True,inplace=True)
df.head()

,Non_Mal,Mal,Ground_Truth
9d7e73653b810f3ffc571bff0d2a406c89b83ddab1b59bb57fb8dde5d162ab54,Non_Malicious,Non_Malicious,Non_Malicious
27e0ad91577b59f33fd3c29e58aa141246f082aefc2948bd0e7a4cbde988c1e2,Non_Malicious,Non_Malicious,Non_Malicious
ff407dccbfa1f1880aa0527075f17c7ac7b43bd15504b6683acbe158f7b9fafa,Non_Malicious,Malicious,Non_Malicious
02139334827d6b4e81bb98a242a3d7566478325c347ca4665a212b5fe8e7be68,Non_Malicious,Malicious,Non_Malicious
9717e2dd85d509b769a7fcc51609929c64fa343d325a7671bb1eb2f328275794,Malicious,Malicious,Malicious


In [8]:
print('Model built on Non_malicious Users')
display((pd.crosstab(df.Non_Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------\n\n")

print('Model built on Malicious Users')
display((pd.crosstab(df.Mal,df.Ground_Truth)))
print("-----------------------------------------------------------------------------------------------------")

file_name='Non-parametric/OneClassSVM/Mar/Non_para_combined.csv'
df.to_csv(file_name)

Model built on Non_malicious Users


Ground_Truth,Malicious,Non_Malicious
Non_Mal,,
Malicious,1259,390
Non_Malicious,3528,4397


-----------------------------------------------------------------------------------------------------


Model built on Malicious Users


Ground_Truth,Malicious,Non_Malicious
Mal,,
Malicious,4196,3849
Non_Malicious,591,938


-----------------------------------------------------------------------------------------------------


## Combined Features using KDE

In [9]:
col=train.columns[:-1]

In [10]:
#### Non-Malicious Users

X_train=train[train.Target==0][col].values
Y_train=train[train.Target==0]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

non_mal=np.exp(kde.score_samples(X_test))

#### Malicious Users

X_train=train[train.Target==1][col].values
Y_train=train[train.Target==1]['Target'].values

scaler=StandardScaler()
scaled_X=scaler.fit_transform(X_train)
X_test=scaler.transform(z[col].values)

kde = KernelDensity(kernel='gaussian', bandwidth=0.2).fit(scaled_X)

mal=np.exp(kde.score_samples(X_test))

df=pd.DataFrame({'Non_Mal':non_mal,'Mal':mal,'Ground_Truth':Y_test.reshape(1,-1)[0]})
df['Factor']=df['Non_Mal']/df['Mal']
df['Factor']=df['Factor'].apply(lambda x: 'Non_Mal' if x>1 else 'Mal')
df['Ground_Truth']=df['Ground_Truth'].apply(lambda x: 'Malicious' if x==1 else 'Non_Malicious')

display(pd.crosstab(df.Ground_Truth,df.Factor))
print("-----------------------------------------------------------------------------------------------------")
print("-----------------------------------------------------------------------------------------------------")
file_name='Non-parametric/KDE/Mar/Non_para_combined.csv'

df.set_index(z.index.values,drop=True,inplace=True)

df.to_csv(file_name)

Factor,Mal,Non_Mal
Ground_Truth,,
Malicious,3799,988
Non_Malicious,2210,2577


-----------------------------------------------------------------------------------------------------
-----------------------------------------------------------------------------------------------------


### KDE for all three months

In [58]:
df_Jan=pd.read_csv('Non-parametric/KDE/Jan/Non_para_combined.csv',index_col='Unnamed: 0')
df_Feb=pd.read_csv('Non-parametric/KDE/Feb/Non_para_combined.csv',index_col='Unnamed: 0')
df_Mar=pd.read_csv('Non-parametric/KDE/Mar/Non_para_combined.csv',index_col='Unnamed: 0')

df_Jan.columns=[str(col)+'_Jan' for col in df_Jan.columns]
df_Feb.columns=[str(col)+'_Feb' for col in df_Feb.columns]
df_Mar.columns=[str(col)+'_Mar' for col in df_Mar.columns]

df=df_Mar.merge(df_Feb, how='outer', left_index=True, right_index=True)

df=df.merge(df_Jan, how='outer', left_index=True, right_index=True)

df.sort_values(by='Ground_Truth_Mar',inplace=True)

display(df.head())
df.to_csv('Non-parametric/KDE/Final_KDE.csv',index=True)

,Non_Mal_Mar,Mal_Mar,Ground_Truth_Mar,Factor_Mar,Non_Mal_Feb,Mal_Feb,Ground_Truth_Feb,Factor_Feb,Non_Mal_Jan,Mal_Jan,Ground_Truth_Jan,Factor_Jan
fffcd63f195ef91b9de3aa2586d02a690ffae891c8b82c94ad84181e9550ffa1,1.998163e+06,7.540636e+04,Malicious,Non_Mal,8.644244e+07,2.846665e+06,Non_Malicious,Non_Mal,1.290768e+09,1.278255e+07,Non_Malicious,Non_Mal
6915929f8f9709e8ae61055d2d2d6c1dd77b2513d0b1f4c12309ddd6856748f4,2.861844e-05,7.386665e-04,Malicious,Mal,1.334927e-02,6.928974e+04,Non_Malicious,Mal,1.495205e+02,1.543484e+05,Non_Malicious,Mal
ccf0f322a0a868f989c11b410479ad035615709051ee3ec01274636a912186a3,3.060318e-06,1.976554e+03,Malicious,Mal,2.669185e-02,2.709174e+03,Non_Malicious,Mal,2.165945e+04,2.018594e+05,Non_Malicious,Mal
ccf32affad43675ab67329cff2bb004cf67dcca9ffa8ac6814c08633789a228c,1.407571e-82,5.390344e-13,Malicious,Mal,2.522715e-56,1.151603e-04,Malicious,Mal,1.372291e-36,6.270303e-02,Malicious,Mal
68ffda988d852e5644a4779f11d15d2ac60f1e9232b91ea786752a474b23582e,5.367333e-22,2.147999e+01,Malicious,Mal,1.199522e-14,5.451818e+01,Malicious,Mal,3.560685e-05,1.829723e+02,Non_Malicious,Mal


### OneClassSVM for all three months

In [60]:
df_Jan=pd.read_csv('Non-parametric/OneClassSVM/Jan/Non_para_combined.csv',index_col='Unnamed: 0')
df_Feb=pd.read_csv('Non-parametric/OneClassSVM/Feb/Non_para_combined.csv',index_col='Unnamed: 0')
df_Mar=pd.read_csv('Non-parametric/OneClassSVM/Mar/Non_para_combined.csv',index_col='Unnamed: 0')

df_Jan.columns=[str(col)+'_Jan' for col in df_Jan.columns]
df_Feb.columns=[str(col)+'_Feb' for col in df_Feb.columns]
df_Mar.columns=[str(col)+'_Mar' for col in df_Mar.columns]

df=df_Mar.merge(df_Feb, how='outer', left_index=True, right_index=True)

df=df.merge(df_Jan, how='outer', left_index=True, right_index=True)

df.sort_values(by='Ground_Truth_Mar',inplace=True)

display(df.head())
df.to_csv('Non-parametric/OneClassSVM/Final_OneClassSVM.csv',index=True)

,Non_Mal_Mar,Mal_Mar,Ground_Truth_Mar,Non_Mal_Feb,Mal_Feb,Ground_Truth_Feb,Non_Mal_Jan,Mal_Jan,Ground_Truth_Jan
fffcd63f195ef91b9de3aa2586d02a690ffae891c8b82c94ad84181e9550ffa1,Non_Malicious,Malicious,Malicious,Non_Malicious,Malicious,Non_Malicious,Non_Malicious,Malicious,Non_Malicious
6915929f8f9709e8ae61055d2d2d6c1dd77b2513d0b1f4c12309ddd6856748f4,Non_Malicious,Malicious,Malicious,Non_Malicious,Malicious,Non_Malicious,Non_Malicious,Malicious,Non_Malicious
ccf0f322a0a868f989c11b410479ad035615709051ee3ec01274636a912186a3,Non_Malicious,Malicious,Malicious,Non_Malicious,Malicious,Non_Malicious,Non_Malicious,Malicious,Non_Malicious
ccf32affad43675ab67329cff2bb004cf67dcca9ffa8ac6814c08633789a228c,Malicious,Malicious,Malicious,Malicious,Malicious,Malicious,Malicious,Malicious,Malicious
68ffda988d852e5644a4779f11d15d2ac60f1e9232b91ea786752a474b23582e,Non_Malicious,Malicious,Malicious,Non_Malicious,Malicious,Malicious,Non_Malicious,Malicious,Non_Malicious
